<a href="https://colab.research.google.com/github/the-faisalahmed/Optimization/blob/main/Spot_It.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

In [ ]:
!pip install pyomo
from pyomo.environ import *
from pyomo.opt import SolverFactory
from pyomo.util.infeasible import log_infeasible_constraints
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def spot_it(N):
    global model
    model = ConcreteModel()

    # Setting indices and variables
    model.I = RangeSet(0,N**2 + N)
    model.J = RangeSet(0,N**2 + N)
    model.X = Var(model.I, model.J, domain = Binary)

    # Rows should only have N+1 symbols
    def cons_rule1(model, i):
        return sum(model.X[(i,j)] for j in model.J) == N+1
    model.cons1 = Constraint(model.I, rule = cons_rule1)

    # Each 2 cards should only have 1 similar symbol
    model.cons = ConstraintList()
    for i in model.I:
        for k in list(range(i+1,N**2+N+1)):
            model.cons.add(sum(model.X[(i,j)]*model.X[(k,j)] for j in model.J) == 1)

    model.Obj = Objective(expr= 1)

    #from pyomo.contrib.latex_printer import latex_printer
    #print(latex_printer(model))

    # Solving
    opt = SolverFactory('ipopt')
    result = opt.solve(model)

    # Checking Status
    from pyomo.opt import SolverStatus
    if (result.solver.status == SolverStatus.ok) and \
        (result.solver.termination_condition == TerminationCondition.optimal):
        # Do something when the solution in optimal and feasible
        print('Solution is Optimal')
    elif (result.solver.termination_condition == TerminationCondition.infeasible):
        # Do something when model in infeasible
        print('Solution is Infeasible')
    else:
            # Something else is wrong
        print("Solver Status:",  result.solver.status)

    # Solve time
    print('Solve Time: ', result.solver.wallclock_time)

    vals = []
    for i,j in np.array(list(model.X)):
        vals.append(model.X[(i,j)].value)
    vals = np.reshape(vals,(N**2+N+1,N**2+N+1))
    vals = vals*list(range(1,N**2+N+2))

    answer = vals[vals>0]
    answer = np.reshape(answer,(N**2+N+1,N+1))

    print('Solution at N = ',N,'\n',answer)

In [ ]:
spot_it(N=3)

Solution is Optimal
Solve Time:  <undefined>
Solution at N =  3 
 [[ 1.  2. 12. 13.]
 [ 1.  7.  8. 11.]
 [ 4.  5. 11. 13.]
 [ 2.  5.  8. 10.]
 [ 7.  9. 10. 13.]
 [ 4.  8.  9. 12.]
 [ 2.  6.  9. 11.]
 [ 5.  6.  7. 12.]
 [ 3. 10. 11. 12.]
 [ 2.  3.  4.  7.]
 [ 1.  3.  5.  9.]
 [ 3.  6.  8. 13.]
 [ 1.  4.  6. 10.]]
